In [140]:
def vars(a, axis=None):
    """ Variance of sparse matrix a
    var = mean(a**2) - mean(a)**2
    """
    a_squared = a.copy()
    a_squared.data **= 2
    return a_squared.mean(axis) - np.square(a.mean(axis))

In [1]:
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import re 

In [3]:
dirc = r'D:\studying\UIUC courses\STAT542\project\Project3\{}'

In [4]:
file = dirc.format('alldata.tsv')

In [6]:
sentiment = []
review = []
cleaner = re.compile('<.*?>') 

In [7]:
with open(file, encoding='UTF-8') as f:
    lines = f.readlines()
    for i in range(1, len(lines)):
        uid, senti, sc, rev = lines[i].split('\t')
        sentiment.append(int(senti))
        rev = re.sub(cleaner, '', rev)
        rev = rev.strip().strip('"')
        review.append(re.sub(cleaner, '', rev))

In [8]:
alldata = pd.DataFrame({'sentiment': sentiment, 'review': review})

In [9]:
del sentiment, review

In [80]:
alldata 

,sentiment,review,indi
0,1,Naturally in a film who's main themes are of m...,True
1,0,This movie is a disaster within a disaster fil...,False
2,0,"All in all, this is a movie for kids. We saw i...",False
3,0,Afraid of the Dark left me with the impression...,False
4,1,A very accurate depiction of small time mob li...,True
...,...,...,...
49995,0,It seems like more consideration has gone into...,False
49996,0,I don't believe they made this film. Completel...,False
49997,0,"Guy is a loser. Can't get girls, needs to buil...",False
49998,0,This 30 minute documentary Buñuel made in the ...,False


In [ ]:
alldata['review'].values[0]

In [77]:
Y = alldata['sentiment']

In [43]:
stop_words = ["i", "me", "my", "myself", 
               "we", "our", "ours", "ourselves", 
               "you", "your", "yours", 
               "their", "they", "his", "her", 
               "she", "he", "a", "an", "and",
               "is", "was", "are", "were", 
               "him", "himself", "has", "have", 
               "it", "its", "the", "us"]

In [56]:
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1, 4), min_df=0.001, max_df=0.5)

corpus = alldata['review'].values

X = vectorizer.fit_transform(corpus)

In [150]:
X.shape

(50000, 31358)

In [151]:
size = X.shape[1]

summ = np.zeros((size, 4))

# mean_1
summ[:,0] = np.array(X[alldata['indi']].mean(axis=0)).reshape(-1)
# var_1 
summ[:,1] = np.array(vars(X[alldata['indi']], axis=0)).reshape(-1)

# mean_2
summ[:,2] = np.array(X[~alldata['indi']].mean(axis=0)).reshape(-1)
# var_2
summ[:,3] = np.array(vars(X[~alldata['indi']], axis=0)).reshape(-1)

In [157]:
summ 

array([[0.00412   , 0.00690303, 0.0042    , 0.00482236],
       [0.01092   , 0.01984075, 0.01384   , 0.02348845],
       [0.001     , 0.001559  , 0.002     , 0.002716  ],
       ...,
       [0.00696   , 0.01515156, 0.02596   , 0.09200608],
       [0.0058    , 0.00744636, 0.0054    , 0.00657084],
       [0.00084   , 0.00115929, 0.00204   , 0.00299584]])

In [160]:
n1 = sum(alldata['sentiment'])
n2 = len(alldata) - n1

In [163]:
tstat = (summ[:,0] - summ[:,2]) / np.sqrt(summ[:,1] / n1 + summ[:,3] / n2)

In [164]:
tstat

array([-0.11681441, -2.21800479, -2.41825417, ..., -9.17723899,
        0.53419444, -2.94346446])

In [169]:
idx = np.argsort(-np.abs(tstat))[:2000]

In [180]:
tstat[idx] >=0

array([False, False,  True, ...,  True,  True,  True])

In [184]:
pos_idx = idx[tstat[idx] >= 0]
neg_idx = idx[tstat[idx] < 0]

In [176]:
words = np.array(vectorizer.get_feature_names(), dtype='str')

In [189]:
pos_words = words[pos_idx]
neg_words = words[neg_idx]

In [190]:
len(pos_words)

733

In [191]:
len(neg_words)

1267

In [196]:
dirc

'D:\\studying\\UIUC courses\\STAT542\\project\\Project3\\{}'

In [224]:
with open(dirc.format('words.txt'), mode='w', encoding='UTF-8') as f:
    for w in pos_words:
        f.write(w + '\n')
    for w in neg_words:
        f.write(w + '\n')  

In [225]:
corpus = None
with open(dirc.format('words.txt'), encoding='UTF-8') as f:
    corpus = f.readlines() 

In [226]:
corpus = [w.strip() for w in corpus]

In [241]:
voca = dict()

In [243]:
for idx, term in enumerate(corpus):
    voca[term] = idx

In [245]:
vectorizer = CountVectorizer(vocabulary=voca)

In [246]:
len(vectorizer.get_feature_names())

2000

In [ ]:
vectorizer.get_feature_names()

In [ ]:
vectorizer.vocabulary_

In [255]:
X = vectorizer.transform(alldata['review'].values).toarray()

In [261]:
X.mean(axis=1)

array([0.011, 0.02 , 0.016, ..., 0.016, 0.015, 0.018])